In [ ]:
import datetime
import requests
import json
import re
import time
from json.decoder import JSONArray

url = "https://api.pushshift.io/reddit/search"

Use the cell below to find convert epoch to datetime and vice versa

In [ ]:
# print(datetime.datetime.fromtimestamp(1607295056))
# print(int(datetime.datetime(2020, 3, 1).timestamp()))

In [ ]:
def fetchObjects(**kwargs):
    
    # Default paramenters. Change as necessary.
    # before: end date (1/12/2020)
    params = {
        "sorted_type": "created_utc",
        "sort": "asc",
        "size": "1000",
        "before": "1606752000"
        }
    mode = kwargs['mode']
    # remove 'mode' from params
    del kwargs['mode']
    # Add additional parameters based on function arguments
    for key, value in kwargs.items():
        params[key] = value

    # sample url: "https://api.pushshift.io/reddit/search/submission/?subreddit=coronavirus&sort=desc&after=1577808000&before=1580486400&size=1000"
    
    loop = True
    while loop:
        # Perform API request
        r = requests.get(f'{url}/{mode}/', params=params, timeout=60)
        # print(r.url)
        if r.status_code != 200:
            print(r.status_code)
            print("Retrying...")
        else:
            # successful (200), loop = false and process data
            loop = False
    else:
        response = json.loads(r.text)
        data = response['data']
        sorted_data_by_id = sorted(data, key=lambda x: int(x['id'],36))
        return sorted_data_by_id

In [ ]:
# Resume from last json object

def resumeFromLast(**kwargs):
    data = [json.loads(line) for line in open('file.json', 'r')]
    last_submission_date = data[-1]['created_utc']
    extract_reddit_data(resume = last_submission_date, **kwargs)

In [ ]:
def extract_reddit_data(**kwargs):
    # kwargs: subreddit
    
    if 'resume' in kwargs:
        start = kwargs['resume']
        del kwargs['resume']
        print(f"Resuming from {datetime.datetime.fromtimestamp(start)}")
    else:
        # initialise start timestamp
        start = int(datetime.datetime(2020, 1, 1).timestamp())
    
    max_id = 0

    # Open file for JSON output
    file = open("file.json", "a")

    while 1: 
        nothing_processed = True
        objects = fetchObjects(**kwargs, after=start)
        print(f"Retrieving data from {datetime.datetime.fromtimestamp(start)}")
        
        for object in objects:
            id = int(object['id'],36)
            if id > max_id:
                nothing_processed = False
                created_utc = object['created_utc']
                max_id = id
                if created_utc > start:
                    start = created_utc
                # Output JSON data to the opened file
                file.write(json.dumps(object,sort_keys=True,ensure_ascii=True) + "\n")

        # Exit if nothing happened
        if nothing_processed: return
        start -= 1

        # Sleep a little before the next function call
        time.sleep(.5)

## Run the cell below to start

In [ ]:
# Manual resume
extract_reddit_data(subreddit='coronavirus', mode='submission')

## Run the cell below to resume

In [ ]:
# extract_reddit_data(subreddit='coronavirus')
resumeFromLast(subreddit='coronavirus', mode="submission")